In [226]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [227]:
! pip install openpyxl

Make the copy of dataset to avoid loss of original data.
Check if there is no null value in the data and display the first five columns.

In [228]:
df = pd.read_excel('/kaggle/input/date-fruit-datasets/Date_Fruit_Datasets/Date_Fruit_Datasets.xlsx')
df1 = df.copy()
df.info(), df.head()

Dataset's stats

In [229]:
df.describe()

In [230]:
df.Class.value_counts()

In [231]:
df.duplicated().sum()

**OBSERVATIONS**
* From above outputs, we can consider the data is clean.
* We can move further to find co-relation between features.

# **Correlation of Features**

In [232]:
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')

In [233]:
x = df.drop('Class', axis = 1)
lb = LabelEncoder()
y = lb.fit_transform(df['Class'])

x.shape, y.shape

In [234]:
x_train,x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)
print (len(x_train),len(x_test),len(y_train),len(y_test))

In [235]:
model_type = [RandomForestClassifier(), SVC(), XGBClassifier()]

In [236]:
for m in model_type:
    pipe = make_pipeline(StandardScaler(), m)
    pipe.fit(x_train, y_train)
    y_predict = pipe.predict(x_test)
    
    print(m,'\nClassificaiton Report\n',classification_report(y_test, y_predict))
    cm = pd.DataFrame(confusion_matrix(y_test, y_predict), index=df['Class'].unique(), columns=df['Class'].unique())
    sns.heatmap(cm, annot=True)
    plt.ylabel('Actual Classes')
    plt.xlabel('Predicted Classes')
    plt.show()

# **COMPARING ACCURACY SCORE OF ALL THREE MODEL**

**RANDOM FOREST**

In [237]:
rfc = RandomForestClassifier(100)
rfc.fit(x_train,y_train)

In [238]:
rfc_y_pred = rfc.predict(x_test)
rfc_y_pred

In [239]:
rfc_acc = accuracy_score(rfc_y_pred, y_test)
rfc_acc

**SUPPORT VECTOR MACHINE**

In [240]:
svc = SVC(C=4)
svc.fit(x_train, y_train)

In [241]:
svc_y_pred = svc.predict(x_test)
svc_y_pred

In [242]:
svc_acc = accuracy_score(y_test, svc_y_pred)
svc_acc

**GRADIENT BOOST (XGBOOST)**

In [243]:
xgbc = XGBClassifier()
xgbc.fit(x_train, y_train)

In [244]:
xgbc_y_pred = xgbc.predict(x_test)
xgbc_y_pred

In [245]:
xgbc_acc = accuracy_score(y_test, xgbc_y_pred)
xgbc_acc

**COMPARING THE ACCURACY VALUES**

In [246]:
algorithms = ['Random Forest Trees','Support Vector Machine','Gradient Boosting']
scores = [rfc_acc, svc_acc, xgbc_acc]
plt.figure(figsize=(10,8))
sns.barplot(algorithms, scores)

# **CONCLUSION**
* FROM ABOVE BAR GRAPH WE CAN CONCLUDE THAT RANDOM FOREST CLASSIFIER WILL WORK BEST ON THE DATASET

Now let's calculate the feature importance and build the model over the important features.

In [247]:
rfc.feature_importances_

In [248]:
feature_imp = pd.DataFrame({'importance': rfc.feature_importances_}, index = x.columns).sort_values('importance')
feature_imp

In [249]:
feature_imp.plot.bar()

In [250]:
lo = feature_imp[feature_imp.importance > 0.02].index

In [251]:
x = x[lo]
x

In [252]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [253]:
clf = RandomForestClassifier(100)
clf.fit(x_train, y_train)
clf.score(x_train,y_train), clf.score(x_test, y_test)

**Here we will use Leave One Feautre Out technique to check whether if it impacts on our accuracy**

In [254]:
results = {}
for col in x.columns:
    temp_x = x.drop(col,axis=1)
    x_train, x_test, y_train, y_test = train_test_split(temp_x,y, test_size = 0.2)
    clf = RandomForestClassifier(100)
    clf.fit(x_train,y_train)
#     print(f'dropped {col}: train_score - {clf.score(X_train,y_train):.3f}, test_score - {clf.score(X_test, y_test):.3f}')
    results[col] = [clf.score(x_train,y_train), clf.score(x_test, y_test)]

In [255]:
pd.DataFrame(results).T.sort_values(by=1)

In [259]:
x = df.drop(['PERIMETER','ALLdaub4RR','CONVEX_AREA','EntropyRG','MeanRG','PERIMETER','Class'], axis=1)
x

In [260]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [266]:
clf = RandomForestClassifier(100)
clf.fit(x_train, y_train)
clf_pred = clf.predict(x_test)
clf_acc = accuracy_score(y_test, clf_pred)
clf_acc

In [267]:
xgbc = XGBClassifier()
xgbc.fit(x_train, y_train)
xgbc_pred = xgbc.predict(x_test)
xgbc_acc = accuracy_score(y_test, xgbc_pred)
xgbc_acc

In [269]:
svc = SVC()
svc.fit(x_train, y_train)
svc_pred = svc.predict(x_test)
svc_acc = accuracy_score(y_test, svc_pred)
svc_acc

In [270]:
algorithms = ['Random Forest Trees','Support Vector Machine','Gradient Boosting']
scores = [rfc_acc, svc_acc, xgbc_acc]
plt.figure(figsize=(10,8))
sns.barplot(algorithms, scores)

# **CONCLUSION**
* As we can see that after applying LOFO technique and dropping the columns with less impotance, the accuracy on our dataset has increased using Random forest and Extra Gradient Boosting.
* Support Vector Machine gives less accuracy as compared to the other even after dropping the columns with less importance.